**Instructions:** You're planning to run the B-TIM for a certain year and forcing. Update CONFIG.py and utils.py appropriately, then run this test. Make sure to edit the ```sys.argv``` variable in the first code cell to the right year and forcing. Then, run the rest of the notebook. 

If you only have files for some months, change the ```enumerate([8,9,10,11,12,1,2,3,4,5,6,7])``` line to only include the month numbers you have (always have to start from August).

In [5]:
import numpy as np
import sys

sys.path.append('../')
import utils

### UPDATE THIS ###
sys.argv = [sys.argv[0], '2019', 'ERA5']
###################

from CONFIG import forcing, data_loc, year



In [7]:
try:
    files = utils.prepare_filenames(forcing, '../'+data_loc, [year, year+1])
except FileNotFoundError:
    print('\n1. Check the "prepare_filenames" function in utils.py.\
           \n2. Ensure the "data_loc" variable in utils.py correctly points to the location where your forcing files are stored.\
           \n3. Check if all monthly files exist for '+str(year)+'-'+str(year+1))
    raise
    
t2m, tp = files[0], files[1]

for mi,m in enumerate([8,9,10,11,12,1,2,3,4,5,6,7]):

    try:
        full_lat, full_lon, t2m_data, tp_data = utils.read_month(m, forcing, t2m[mi], tp[mi])
    except OSError:
        print(utils.month_names_aug[mi]+' ISSUE: File(s) for this month may not exist.')
        raise
    except IndexError:
        print(utils.month_names_aug[mi]+' ISSUE: Check that the latitude and longitude names set in the "latname" and "lonname" dictionaries for this dataset match the name in this month\'s forcing file.')
        raise
        
    ### t2m
    time_step = t2m_data['time'][1]-t2m_data['time'][0]
    time_freq = 24 // time_step

    try:
        utils.t2m_freq[forcing]
    except KeyError:
        print('Add "'+forcing+': '+str(time_freq)+'" to the t2m_freq dictionary in "utils.py"')
        raise
    else:
        if time_freq != utils.t2m_freq[forcing]:
            raise ValueError(utils.month_names_aug[mi]+' ISSUE: The time frequency for this forcing doesn\'t match expected value. Check for consistency across different months and possibly update the t2m_freq dictionary in "utils.py".\
                             \npIt should say "'+forcing+': '+str(time_freq)+'" in the t2m_freq dictionary in "utils.py".')

    ### tp
    time_step = tp_data['time'][1]-tp_data['time'][0]
    time_freq = 24 // time_step

    try:
        utils.tp_freq[forcing]
    except KeyError:
        print('Add "'+forcing+': '+str(time_freq)+'" to the tp_freq dictionary in "utils.py"')
        raise
    else:
        if time_freq != utils.tp_freq[forcing]:
            raise ValueError(utils.month_names_aug[mi]+' ISSUE: The time frequency for this forcing doesn\'t match expected value. Check for consistency across different months and possibly update the tp_freq dictionary in "utils.py".\
                             \npIt should say "'+forcing+': '+str(time_freq)+'" in the tp_freq dictionary in "utils.py".')

    ## test read_day
    try:
        pname = utils.precipname[forcing]
    except KeyError:
        print('Update the name of the precipitation variable in the "precipname" dictionary in utils.py')
        raise
        
    try:
        pname = utils.tempname[forcing]
    except KeyError:
        print('Update the name of the temperature variable in the "tempname" dictionary in utils.py')
        raise


    try:
        a = utils.read_day(forcing, t2m_data, 't2m', 0, np.ones_like(full_lat, dtype='bool'), np.ones_like(full_lon, dtype='bool'))    
        b = utils.read_day(forcing, tp_data, 'tp', 0, np.ones_like(full_lat, dtype='bool'), np.ones_like(full_lon, dtype='bool'))    
    except:
        print(utils.month_names_aug[mi]+' ISSUE: check read_day function in "utils.py" or this month\'s file to ensure the data is loading properly.')
        raise
